In [ ]:
import cv2
import numpy as np
import time
import urllib.request
import os
from collections import deque

In [10]:
algo = "geo"
cam_index = 0
smile_thresh_geo = 0.04
smile_thresh_mar = 0.34
font = cv2.FONT_HERSHEY_SIMPLEX

face_cascade_filename = "haarcascade_frontalface_default.xml"
eye_cascade_filename = "haarcascade_eye.xml"
smile_cascade_filename = "haarcascade_smile.xml"

base_urls = [
    "https://github.com/opencv/opencv/raw/4.x/data/haarcascades/",
    "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/"
]

def download_if_needed(filename):
    if os.path.exists(filename):
        return True
    for base in base_urls:
        url = base + filename
        try:
            print(f"Файл '{filename}' не найден, скачивает из {url}")
            urllib.request.urlretrieve(url, filename)
            print("Файл скачан")
            return True
        except Exception as e:
            print(f"Не удалось скачать из {url}: {e}")
    return False

def load_cascade(filename):
    if os.path.exists(filename):
        cas = cv2.CascadeClassifier(filename)
        if not cas.empty():
            return cas
        else:
            print(f"Внимание: файл {filename} есть, но загрузка не удалась (возможен битый файл)")
    try:
        haar_dir = cv2.data.haarcascades
        path = os.path.join(haar_dir, filename)
        cas = cv2.CascadeClassifier(path)
        if not cas.empty():
            print(f"Загружено из cv2.data.haarcascades: {path}")
            return cas
    except Exception:
        pass
    return cv2.CascadeClassifier() 

download_if_needed(face_cascade_filename)
download_if_needed(eye_cascade_filename)
download_if_needed(smile_cascade_filename)

face_cascade = load_cascade(face_cascade_filename)
eye_cascade = load_cascade(eye_cascade_filename)
smile_cascade = load_cascade(smile_cascade_filename)



# Media pipe

In [11]:
import mediapipe as mp

In [12]:

cam_index = 0

smile_thresh_geo = 0.028
smile_thresh_mar = 0.34
yawn_mar_thresh = 0.52
lift_thresh = 0.05

slouch_thresh_z = -0.5      
shoulder_tilt_thresh = 0.07 

font = cv2.FONT_HERSHEY_SIMPLEX
min_vis = 0.6  

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    model_complexity=1,
    enable_segmentation=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

LM_MOUTH_LEFT = 61
LM_MOUTH_RIGHT = 291
LM_UPPER_INNER = 13
LM_LOWER_INNER = 14
LM_INNER_LEFT = 78
LM_INNER_RIGHT = 308
LM_LEFT_EYE_UPPER = 159
LM_LEFT_EYE_LOWER = 145
LM_LEFT_EYE_INNER = 133
LM_LEFT_EYE_OUTER = 33
LM_RIGHT_EYE_UPPER = 386
LM_RIGHT_EYE_LOWER = 374
LM_RIGHT_EYE_INNER = 362
LM_RIGHT_EYE_OUTER = 263

LM_FACE_LEFT_EAR = 234
LM_FACE_RIGHT_EAR = 454

POSE_LEFT_SHOULDER = 11
POSE_RIGHT_SHOULDER = 12
POSE_LEFT_ELBOW = 13
POSE_RIGHT_ELBOW = 14
POSE_LEFT_WRIST = 15
POSE_RIGHT_WRIST = 16
POSE_LEFT_HIP = 23
POSE_RIGHT_HIP = 24
POSE_LEFT_EAR = 7    
POSE_RIGHT_EAR = 8  

def to_px(landmark, w, h):
    return np.array([landmark.x * w, landmark.y * h], dtype=np.float32)

def norm_point_line_distance(p, a, b):
    ab = b - a
    ap = p - a
    cross = np.abs(np.cross(ab, ap))
    denom = np.linalg.norm(ab) + 1e-8
    return (cross / denom) / (denom + 1e-8)

def mouth_mar(landmarks, w, h):
    A = to_px(landmarks[LM_UPPER_INNER], w, h)
    B = to_px(landmarks[LM_LOWER_INNER], w, h)
    L = to_px(landmarks[LM_INNER_LEFT], w, h)
    R = to_px(landmarks[LM_INNER_RIGHT], w, h)
    height = np.linalg.norm(B - A)
    width = np.linalg.norm(R - L) + 1e-8
    return height / width

def mouth_corner_lift(landmarks, w, h):
    L = to_px(landmarks[LM_MOUTH_LEFT], w, h)
    R = to_px(landmarks[LM_MOUTH_RIGHT], w, h)
    C_up = to_px(landmarks[LM_UPPER_INNER], w, h)
    C_dn = to_px(landmarks[LM_LOWER_INNER], w, h)
    C = (C_up + C_dn) / 2.0
    corner_y = (L[1] + R[1]) / 2.0
    width = np.linalg.norm(R - L) + 1e-8
    return (C[1] - corner_y) / width

def smile_geo_metric(landmarks, w, h):
    L = to_px(landmarks[LM_MOUTH_LEFT], w, h)
    R = to_px(landmarks[LM_MOUTH_RIGHT], w, h)
    C = to_px(landmarks[LM_UPPER_INNER], w, h)
    return norm_point_line_distance(C, L, R)

def are_eyes_open(landmarks, w, h, threshold=0.2):
    left_v = np.linalg.norm(to_px(landmarks[LM_LEFT_EYE_LOWER], w, h) - to_px(landmarks[LM_LEFT_EYE_UPPER], w, h))
    left_h = np.linalg.norm(to_px(landmarks[LM_LEFT_EYE_OUTER], w, h) - to_px(landmarks[LM_LEFT_EYE_INNER], w, h))
    right_v = np.linalg.norm(to_px(landmarks[LM_RIGHT_EYE_LOWER], w, h) - to_px(landmarks[LM_RIGHT_EYE_UPPER], w, h))
    right_h = np.linalg.norm(to_px(landmarks[LM_RIGHT_EYE_OUTER], w, h) - to_px(landmarks[LM_RIGHT_EYE_INNER], w, h))
    left_ear = left_v / (left_h + 1e-8)
    right_ear = right_v / (right_h + 1e-8)
    return ((left_ear + right_ear) / 2.0) > threshold

def check_posture(pose_lms, face_lms, w, h):
    if pose_lms is None or face_lms is None:
        return "UNKNOWN", (200, 200, 200)

    p_lms = pose_lms.landmark
    
    left_shoulder = p_lms[POSE_LEFT_SHOULDER]
    right_shoulder = p_lms[POSE_RIGHT_SHOULDER]
    left_ear = face_lms[LM_FACE_LEFT_EAR]
    right_ear = face_lms[LM_FACE_RIGHT_EAR]

    if (left_shoulder.visibility < min_vis or right_shoulder.visibility < min_vis):
        return "UNKNOWN", (200, 200, 200)

    shoulder_y_diff = abs(left_shoulder.y - right_shoulder.y)
    if shoulder_y_diff > shoulder_tilt_thresh:
        return "TILTED", (0, 165, 255)

    avg_shoulder_z = (left_shoulder.z + right_shoulder.z) / 2.0
    avg_ear_z = (left_ear.z + right_ear.z) / 2.0
    z_diff = avg_shoulder_z - avg_ear_z
    
    if z_diff < slouch_thresh_z:
        return "SLOUCHED", (0, 0, 255)

    return "GOOD", (0, 255, 0)

def is_hand_raised(pose_lms, left=True):
    if pose_lms is None: 
        return False
    lms = pose_lms.landmark
    
    sh_idx, el_idx, wr_idx = (POSE_LEFT_SHOULDER, POSE_LEFT_ELBOW, POSE_LEFT_WRIST) if left else (POSE_RIGHT_SHOULDER, POSE_RIGHT_ELBOW, POSE_RIGHT_WRIST)
    
    sh, el, wr = lms[sh_idx], lms[el_idx], lms[wr_idx]
    
    if (sh.visibility < min_vis) or (el.visibility < min_vis) or (wr.visibility < min_vis):
        return False
    
    return (wr.y < el.y) and (el.y < sh.y - 0.03)

clap_state = 'APART'
clap_count = 0
clap_distance_threshold = 0.15

def detect_clap(pose_lms, current_clap_state, current_clap_count):
    if pose_lms is None: 
        return 'APART', current_clap_count, False

    lms = pose_lms.landmark
    left_wrist = lms[POSE_LEFT_WRIST]
    right_wrist = lms[POSE_RIGHT_WRIST]
    
    if (left_wrist.visibility < min_vis) or (right_wrist.visibility < min_vis):
        return current_clap_state, current_clap_count, False
    
    distance = abs(left_wrist.x - right_wrist.x)
    clap_detected_this_frame = False
    
    if distance < clap_distance_threshold:
        if current_clap_state == 'APART':
            current_clap_count += 1
            clap_detected_this_frame = True
            new_clap_state = 'TOGETHER'
        else:
            new_clap_state = 'TOGETHER'
    else:
        new_clap_state = 'APART'
        
    return new_clap_state, current_clap_count, clap_detected_this_frame

def annotate_frame(frame, face_lms, pose_lms, smile, eyes, hands, posture, claps):
    h, w = frame.shape[:2]
    
    cv2.putText(frame, "SMILE" if smile else "NO SMILE", (10, 60), font, 0.9, (0, 255, 0) if smile else (0, 0, 255), 2)
    cv2.putText(frame, f"LEFT_RAISED: {hands['left']}", (10, 90), font, 0.7, (0, 255, 0) if hands['left'] else (0, 0, 255), 2)
    cv2.putText(frame, f"RIGHT_RAISED: {hands['right']}", (10, 115), font, 0.7, (0, 255, 0) if hands['right'] else (0, 0, 255), 2)
    cv2.putText(frame, "EYES: OPEN" if eyes else "EYES: CLOSED", (10, 140), font, 0.7, (0, 255, 0) if eyes else (0, 0, 255), 2)
    cv2.putText(frame, f"POSTURE: {posture[0]}", (10, 165), font, 0.7, posture[1], 2)
    cv2.putText(frame, f"CLAPS: {claps['count']}", (10, 190), font, 0.9, (0, 255, 255), 2)
    
    if face_lms:
        L = to_px(face_lms[LM_MOUTH_LEFT], w, h).astype(int)
        R = to_px(face_lms[LM_MOUTH_RIGHT], w, h).astype(int)
        A = to_px(face_lms[LM_UPPER_INNER], w, h).astype(int)
        B = to_px(face_lms[LM_LOWER_INNER], w, h).astype(int)
        cv2.line(frame, L, R, (255, 255, 0), 1)
        for p, color in [(L, (0,255,255)), (R, (0,255,255)), (A, (0,140,255)), (B, (0,140,255))]:
            cv2.circle(frame, p, 3, color, -1)
    
    if pose_lms:
        lms = pose_lms.landmark
        pts = {}
        pose_indices = [
            POSE_LEFT_SHOULDER, POSE_RIGHT_SHOULDER, POSE_LEFT_ELBOW, POSE_RIGHT_ELBOW,
            POSE_LEFT_WRIST, POSE_RIGHT_WRIST, POSE_LEFT_HIP, POSE_RIGHT_HIP,
            POSE_LEFT_EAR, POSE_RIGHT_EAR 
        ]
        for idx in pose_indices:
            if lms[idx].visibility > min_vis:
                pts[idx] = (int(lms[idx].x * w), int(lms[idx].y * h))
        
        colorL = (0, 255, 0) if hands['left'] else (0, 0, 255)
        colorR = (0, 255, 0) if hands['right'] else (0, 0, 255)
        
        connections = [
            (POSE_LEFT_SHOULDER, POSE_RIGHT_SHOULDER, posture[1]),
            (POSE_LEFT_HIP, POSE_RIGHT_HIP, posture[1]),
            (POSE_LEFT_SHOULDER, POSE_LEFT_HIP, posture[1]),
            (POSE_RIGHT_SHOULDER, POSE_RIGHT_HIP, posture[1]),
            (POSE_LEFT_SHOULDER, POSE_LEFT_ELBOW, colorL),
            (POSE_LEFT_ELBOW, POSE_LEFT_WRIST, colorL),
            (POSE_RIGHT_SHOULDER, POSE_RIGHT_ELBOW, colorR),
            (POSE_RIGHT_ELBOW, POSE_RIGHT_WRIST, colorR),
            (POSE_LEFT_WRIST, POSE_RIGHT_WRIST, (0, 255, 255) if claps['detected'] else (200, 200, 200))
        ]
        
        for conn in connections:
            if conn[0] in pts and conn[1] in pts:
                cv2.line(frame, pts[conn[0]], pts[conn[1]], conn[2], 2)
        
        for idx, pt in pts.items():
            cv2.circle(frame, pt, 4, (255, 255, 255), -1)

In [13]:
cap = cv2.VideoCapture(cam_index)
if not cap.isOpened():
    raise RuntimeError("Не удалось открыть камеру")

fps_time = time.time()
clap_state = 'APART'
clap_count = 0

while True:
    ok, frame = cap.read()
    if not ok: 
        break
    
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    h, w = frame.shape[:2]

    smiling, eyes_open = False, False
    face_res = face_mesh.process(rgb)
    face_lms = face_res.multi_face_landmarks[0].landmark if face_res.multi_face_landmarks else None
    
    if face_lms:
        mar = mouth_mar(face_lms, w, h)
        lift = mouth_corner_lift(face_lms, w, h)
        d = smile_geo_metric(face_lms, w, h)
        eyes_open = are_eyes_open(face_lms, w, h)
        smiling_geo = (d > smile_thresh_geo) and (lift > lift_thresh) and (mar < yawn_mar_thresh)
        smiling_mar = (mar > smile_thresh_mar) and (lift > lift_thresh)
        smiling = smiling_geo or smiling_mar

    pose_res = pose.process(rgb)
    pose_lms = pose_res.pose_landmarks if pose_res and pose_res.pose_landmarks else None
    
    left_raised = is_hand_raised(pose_lms, left=True)
    right_raised = is_hand_raised(pose_lms, left=False)
    
    posture_result = check_posture(pose_lms, face_lms, w, h)
    
    clap_state, clap_count, clap_detected = detect_clap(pose_lms, clap_state, clap_count)

    annotate_frame(frame, face_lms, pose_lms, smiling, eyes_open, 
                   {'left': left_raised, 'right': right_raised},
                   posture_result,
                   {'count': clap_count, 'detected': clap_detected})
    
    now = time.time()
    fps = 1.0 / max(now - fps_time, 1e-6)
    fps_time = now
    cv2.putText(frame, f"FPS:{fps:.1f}", (10, h - 10), font, 0.6, (180, 180, 180), 1)

    cv2.imshow("Smile + Hands + Claps + Eyes + Posture", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  
        break

cap.release()
cv2.destroyAllWindows()

# DLIB

In [2]:
import dlib
import bz2
import mediapipe as mp


In [14]:

CAM_INDEX = 0
FONT = cv2.FONT_HERSHEY_SIMPLEX
MIN_VISIBILITY = 0.6

def download_predictor(dat_name="shape_predictor_68_face_landmarks.dat"):
    """Скачивает и распаковывает модель dlib, если она отсутствует."""
    bz2_name = dat_name + ".bz2"
    url = "http://dlib.net/files/" + bz2_name
    if os.path.exists(dat_name):
        return True
    if not os.path.exists(bz2_name):
        try:
            print(f"Скачивает {bz2_name}...")
            urllib.request.urlretrieve(url, bz2_name)
            print("Файл скачан.")
        except Exception as e:
            print(f"Не удалось скачать: {e}")
            return False
    try:
        with bz2.BZ2File(bz2_name, 'rb') as f_in, open(dat_name, 'wb') as f_out:
            f_out.write(f_in.read())
        print("Распаковка завершена.")
        return True
    except Exception as e:
        print(f"Ошибка распаковки: {e}")
        return False

if not download_predictor():
    raise SystemExit("КРИТИЧЕСКАЯ ОШИБКА: не удалось загрузить модель dlib.")

dlib_detector = dlib.get_frontal_face_detector()
dlib_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

mp_pose = mp.solutions.pose
pose_detector = mp_pose.Pose(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
def shape_to_np(shape):

    return np.array([(p.x, p.y) for p in shape.parts()], dtype=np.float32)

def are_eyes_open(lms, threshold=0.2):

    if lms is None: return False

    left_v = np.linalg.norm(lms[37] - lms[41])
    left_h = np.linalg.norm(lms[36] - lms[39])
    left_ear = left_v / (left_h + 1e-8)

    right_v = np.linalg.norm(lms[43] - lms[47])
    right_h = np.linalg.norm(lms[45] - lms[42])
    right_ear = right_v / (right_h + 1e-8)

    return (left_ear > threshold) and (right_ear > threshold)

def is_hand_raised(pose_lms, left=True):
    if pose_lms is None: return False
    lms = pose_lms.landmark
    sh, el, wr = (11, 13, 15) if left else (12, 14, 16)
    
    if not all(lms[i].visibility > MIN_VISIBILITY for i in [sh, el, wr]):
        return False
        
 
class SmileDetector:
    def __init__(self, smooth=5, base=25):
        self.metrics = {k: deque(maxlen=smooth) for k in ['d', 'lift', 'mar', 'mwr']}
        self.base_metrics = {k: deque(maxlen=base) for k in ['d', 'lift', 'mwr']}
    
    @staticmethod
    def _median(q): return np.median(np.array(q)) if q else 0.0

    def update(self, lms):
        if lms is None: return False
        
        mouth_width = np.linalg.norm(lms[48] - lms[54]) + 1e-6
        d = np.abs(np.cross(lms[54]-lms[48], lms[62]-lms[48])) / mouth_width
       
        lift = ((lms[62][1]+lms[66][1])/2) - ((lms[48][1]+lms[54][1])/2)

        mar = np.linalg.norm(lms[62]-lms[66]) / (np.linalg.norm(lms[60]-lms[64])+1e-6)

        mwr = mouth_width / (np.linalg.norm(lms[36]-lms[45])+1e-6)

        for k, v in zip(self.metrics.keys(), [d, lift, mar, mwr]): self.metrics[k].append(v)
        smoothed = {k: self._median(v) for k, v in self.metrics.items()}

        if smoothed['mar'] < 0.4 and smoothed['lift'] > -10:
            for k in self.base_metrics.keys(): self.base_metrics[k].append(smoothed[k])
        
        base = {k: self._median(v) for k, v in self.base_metrics.items()}
        
        if smoothed['mar'] > 0.8: return False # Анти-зевок

        if len(self.base_metrics) < 10: return (smoothed['lift'] > 2) and (smoothed['mwr'] > 0.6)
        
        hits = (smoothed['d'] > base['d'] + 1.5) + \
               (smoothed['lift'] > base['lift'] + 2.0) + \
               (smoothed['mwr'] > base['mwr'] * 1.05)
        
        return hits >= 2

class PostureDetector:
    def __init__(self, buffer_size=10, slouch_thresh=0.3, tilt_thresh=0.07):
        self.z_diff_buffer = deque(maxlen=buffer_size)
        self.slouch_thresh, self.tilt_thresh = slouch_thresh, tilt_thresh
    
    def update(self, pose_lms):
        if pose_lms is None or not all(pose_lms.landmark[i].visibility > MIN_VISIBILITY for i in [7,8,11,12]):
            self.z_diff_buffer.clear()
            return "UNKNOWN", (200, 200, 200), 0.0

        p_lms = pose_lms.landmark
        z_diff = ((p_lms[11].z + p_lms[12].z)/2) - ((p_lms[7].z + p_lms[8].z)/2)
        self.z_diff_buffer.append(z_diff)
        
        if len(self.z_diff_buffer) < self.z_diff_buffer.maxlen / 2:
            return "WAITING...", (200, 200, 200), 0.0
            
        smoothed_z = np.median(self.z_diff_buffer)

        if abs(p_lms[11].y - p_lms[12].y) > self.tilt_thresh:
            return "TILTED", (0, 165, 255), smoothed_z
        
        if smoothed_z > self.slouch_thresh:
            return "SLOUCHED", (0, 0, 255), smoothed_z
            
        return "GOOD", (0, 255, 0), smoothed_z

class ClapDetector:
    def __init__(self, distance_px=60, cooldown=0.3):
        self.distance_px, self.cooldown = distance_px, cooldown
        self.state_open = True
        self.last_event_t = 0.0
    
    def update(self, pose_lms, w, h):
        now = time.time()
        if pose_lms is None or not all(pose_lms.landmark[i].visibility > MIN_VISIBILITY for i in [15,16]):
            return False
            
        lms = pose_lms.landmark
        d_px = np.linalg.norm(np.array([(lms[16].x-lms[15].x)*w, (lms[16].y-lms[15].y)*h]))
        hands_close = d_px < self.distance_px
        
        event = False
        if self.state_open and hands_close and (now - self.last_event_t > self.cooldown):
            event, self.state_open, self.last_event_t = True, False, now
        
        if not hands_close:
            self.state_open = True
            
        return event


def draw_landmarks(frame, face_lms, pose_lms, posture_color):
    if face_lms is not None:
        for i in range(68):
            cv2.circle(frame, tuple(face_lms[i].astype(int)), 1, (0, 255, 0), -1)
            
    if pose_lms is not None:
        mp.solutions.drawing_utils.draw_landmarks(
            frame, pose_lms, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_styles.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            connection_drawing_spec=mp.solutions.drawing_styles.DrawingSpec(color=posture_color, thickness=2, circle_radius=2)
        )

def draw_final_status(frame, fps, states):
    h, _ = frame.shape[:2]
    cv2.putText(frame, f"FPS:{fps:.1f}", (10, h - 10), FONT, 0.6, (180, 180, 180), 1)
    
    posture_status, posture_color = states['posture']
    posture_text = f"POSTURE: {posture_status} (Z: {states.get('z_diff',0.0):.2f})"
    
    cv2.putText(frame, "SMILE" if states['smiling'] else "NO SMILE", (10,30), FONT, 0.7, (0,255,0) if states['smiling'] else (0,0,255), 2)
    cv2.putText(frame, f"EYES: {'OPEN' if states['eyes_open'] else 'CLOSED'}", (10,55), FONT, 0.7, (0,255,0) if states['eyes_open'] else (0,0,255), 2)
    cv2.putText(frame, posture_text, (10, 80), FONT, 0.7, posture_color, 2)
    cv2.putText(frame, f"L.HAND: {'UP' if states['left_raised'] else 'DOWN'}", (10,105), FONT, 0.7, (0,255,0) if states['left_raised'] else (200,200,200), 2)
    cv2.putText(frame, f"R.HAND: {'UP' if states['right_raised'] else 'DOWN'}", (10,130), FONT, 0.7, (0,255,0) if states['right_raised'] else (200,200,200), 2)
    cv2.putText(frame, f"CLAPS: {states['clap_count']}", (10,155), FONT, 0.7, (0,255,255), 2)

In [20]:

cap = cv2.VideoCapture(CAM_INDEX)
posture_det = PostureDetector(slouch_thresh=0.3)
smile_det = SmileDetector()
clap_det = ClapDetector()
clap_count = 0
fps_time = time.time()

while True:
    ok, frame = cap.read()
    if not ok: break
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    h, w = frame.shape[:2]

    pose_results = pose_detector.process(rgb_frame)
    pose_landmarks = pose_results.pose_landmarks
    dlib_faces = dlib_detector(rgb_frame, 0)
    face_landmarks = shape_to_np(dlib_predictor(rgb_frame, dlib_faces[0])) if dlib_faces else None

    states = {}
    posture_status, posture_color, z_diff = posture_det.update(pose_landmarks)
    states['posture'], states['z_diff'] = (posture_status, posture_color), z_diff
    states['smiling'] = smile_det.update(face_landmarks)
    states['eyes_open'] = are_eyes_open(face_landmarks)
    states['left_raised'], states['right_raised'] = is_hand_raised(pose_landmarks,True), is_hand_raised(pose_landmarks,False)
    if clap_det.update(pose_landmarks, w, h): clap_count += 1
    states['clap_count'] = clap_count
    
    draw_landmarks(frame, face_landmarks, pose_landmarks, posture_color)
    now = time.time(); fps = 1.0/max(now-fps_time, 1e-6); fps_time = now
    draw_final_status(frame, fps, states)
    
    cv2.imshow("Multi-Event Detector", frame)
    if cv2.waitKey(1) & 0xFF == 27: break

cap.release()
cv2.destroyAllWindows()

# MTCNN

In [27]:
USE_FPT = False
FPT_MTCNN = None
IPAZC_MTCNN = None

try:
    from facenet_pytorch import MTCNN as FPT_MTCNN
    USE_FPT = True
except Exception:
    pass

try:
    from mtcnn.mtcnn import MTCNN as IPAZC_MTCNN
except Exception:
    IPAZC_MTCNN = None

def init_mtcnn():
    fpt = FPT_MTCNN(keep_all=True) if (USE_FPT and FPT_MTCNN is not None) else None
    ipz = IPAZC_MTCNN() if (IPAZC_MTCNN is not None) else None
    return fpt, ipz


def mtcnn_detect(fpt, ipz, frame_bgr):
    dets = []
    if fpt is not None:
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        boxes, probs, landmarks = fpt.detect(rgb, landmarks=True)
        if boxes is not None:
            for i, b in enumerate(boxes):
                if probs is None or probs[i] is None:
                    continue
                x1, y1, x2, y2 = b.astype(int)
                w = int(x2 - x1); h = int(y2 - y1)
                lm = landmarks[i]
                kp = {
                    "left_eye":   (int(lm[0][0]), int(lm[0][1])),
                    "right_eye":  (int(lm[1][0]), int(lm[1][1])),
                    "nose":       (int(lm[2][0]), int(lm[2][1])),
                    "mouth_left": (int(lm[3][0]), int(lm[3][1])),
                    "mouth_right":(int(lm[4][0]), int(lm[4][1])),
                }
                dets.append({"box": [int(x1), int(y1), w, h], "keypoints": kp, "confidence": float(probs[i])})
    if (not dets) and (ipz is not None):
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        raw = ipz.detect_faces(rgb)
        for r in raw:
            x, y, w, h = r["box"]
            kp = r.get("keypoints", {})
            dets.append({"box": [int(x), int(y), int(w), int(h)],
                         "keypoints": {k: (int(v[0]), int(v[1])) for k, v in kp.items()},
                         "confidence": float(r.get("confidence", 0.0))})
    return dets

def draw_mtcnn_detections(frame, detections):
    for det in detections:
        x, y, w, h = det["box"]
        kp = det.get("keypoints", {})
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        for name in ("left_eye", "right_eye", "nose", "mouth_left", "mouth_right"):
            if name in kp and kp[name] is not None:
                cx, cy = int(kp[name][0]), int(kp[name][1])
                cv2.circle(frame, (cx, cy), 3, (0, 255, 0), -1)
                cv2.putText(frame, name, (cx+4, cy-4), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (50, 255, 50), 1)


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

LM_MOUTH_LEFT = 61; LM_MOUTH_RIGHT = 291; LM_UPPER_INNER = 13; LM_LOWER_INNER = 14
LM_INNER_LEFT = 78; LM_INNER_RIGHT = 308
LM_LEFT_EYE_UPPER = 159; LM_LEFT_EYE_LOWER = 145; LM_LEFT_EYE_INNER = 133; LM_LEFT_EYE_OUTER = 33
LM_RIGHT_EYE_UPPER = 386; LM_RIGHT_EYE_LOWER = 374; LM_RIGHT_EYE_INNER = 362; LM_RIGHT_EYE_OUTER = 263

def to_px(landmark, w, h):
    return np.array([landmark.x * w, landmark.y * h], dtype=np.float32)

def mouth_mar(lms, w, h):
    A = to_px(lms[LM_UPPER_INNER], w, h)
    B = to_px(lms[LM_LOWER_INNER], w, h)
    L = to_px(lms[LM_INNER_LEFT], w, h)
    R = to_px(lms[LM_INNER_RIGHT], w, h)
    height = np.linalg.norm(B - A)
    width = np.linalg.norm(R - L) + 1e-8
    return float(height / width)

def mouth_corner_lift(lms, w, h):
    L = to_px(lms[LM_MOUTH_LEFT], w, h)
    R = to_px(lms[LM_MOUTH_RIGHT], w, h)
    C_up = to_px(lms[LM_UPPER_INNER], w, h)
    C_dn = to_px(lms[LM_LOWER_INNER], w, h)
    C = (C_up + C_dn) / 2.0
    corner_y = (L[1] + R[1]) / 2.0
    width = np.linalg.norm(R - L) + 1e-8
    return float((C[1] - corner_y) / width)

def smile_geo_metric(lms, w, h):
    L = to_px(lms[LM_MOUTH_LEFT], w, h)
    R = to_px(lms[LM_MOUTH_RIGHT], w, h)
    C = to_px(lms[LM_UPPER_INNER], w, h)
    ab = R - L; ap = C - L
    cross = np.abs(np.cross(ab, ap))
    denom = np.linalg.norm(ab) + 1e-8
    return float((cross / denom) / (denom + 1e-8))

def is_smile_facemesh(lms, w, h):
    d = smile_geo_metric(lms, w, h)
    lift = mouth_corner_lift(lms, w, h)
    mar = mouth_mar(lms, w, h)
    smile_main = (d > 0.016) and (lift > 0.050) and (mar < 0.60)
    smile_teeth = (mar > 0.32) and (lift > 0.050)
    smiling = bool(smile_main or smile_teeth)
    if (mar > 0.70) and (lift < 0.03):
        smiling = False
    return smiling, {"d": d, "lift": lift, "mar": mar}

def are_eyes_open(lms, w, h, threshold=0.2):
    left_v = np.linalg.norm(to_px(lms[LM_LEFT_EYE_LOWER], w, h) - to_px(lms[LM_LEFT_EYE_UPPER], w, h))
    left_h = np.linalg.norm(to_px(lms[LM_LEFT_EYE_OUTER], w, h) - to_px(lms[LM_LEFT_EYE_INNER], w, h))
    left_ear = left_v / (left_h + 1e-8)
    
    right_v = np.linalg.norm(to_px(lms[LM_RIGHT_EYE_LOWER], w, h) - to_px(lms[LM_RIGHT_EYE_UPPER], w, h))
    right_h = np.linalg.norm(to_px(lms[LM_RIGHT_EYE_OUTER], w, h) - to_px(lms[LM_RIGHT_EYE_INNER], w, h))
    right_ear = right_v / (right_h + 1e-8)
    
    return (left_ear > threshold) and (right_ear > threshold)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    model_complexity=2,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

POSE_LEFT_SHOULDER = 11; POSE_RIGHT_SHOULDER = 12
POSE_LEFT_ELBOW = 13; POSE_RIGHT_ELBOW = 14
POSE_LEFT_WRIST = 15; POSE_RIGHT_WRIST = 16
POSE_LEFT_EAR = 7; POSE_RIGHT_EAR = 8

raise_margin = 0.03
min_vis = 0.6
font = cv2.FONT_HERSHEY_SIMPLEX

class PostureDetector:
    def __init__(self, buffer_size=10, slouch_thresh=0.3, tilt_thresh=0.07):
        self.z_diff_buffer = deque(maxlen=buffer_size)
        self.slouch_thresh = slouch_thresh
        self.tilt_thresh = tilt_thresh
    
    def update(self, pose_lms):
        if pose_lms is None or not all(pose_lms.landmark[i].visibility > min_vis for i in [7,8,11,12]):
            self.z_diff_buffer.clear()
            return "UNKNOWN", (200, 200, 200), 0.0
        
        p_lms = pose_lms.landmark
        z_diff = ((p_lms[11].z + p_lms[12].z)/2) - ((p_lms[7].z + p_lms[8].z)/2)
        self.z_diff_buffer.append(z_diff)
        
        if len(self.z_diff_buffer) < self.z_diff_buffer.maxlen / 2:
            return "WAITING...", (200, 200, 200), 0.0
        
        smoothed_z = np.median(self.z_diff_buffer)
        
        if abs(p_lms[11].y - p_lms[12].y) > self.tilt_thresh:
            return "TILTED", (0, 165, 255), smoothed_z
        
        if smoothed_z > self.slouch_thresh:
            return "SLOUCHED", (0, 0, 255), smoothed_z
        
        return "GOOD", (0, 255, 0), smoothed_z

def pose_point_xy(lm, w, h):
    return np.array([lm.x * w, lm.y * h], dtype=np.float32)

def both_vis_ok(pose_lms, idx_a, idx_b):
    la = pose_lms.landmark[idx_a]; lb = pose_lms.landmark[idx_b]
    return (la.visibility >= min_vis) and (lb.visibility >= min_vis)

def is_hand_raised(pose_lms, left=True):
    if pose_lms is None:
        return False
    sh_idx, wr_idx = (POSE_LEFT_SHOULDER, POSE_LEFT_WRIST) if left else (POSE_RIGHT_SHOULDER, POSE_RIGHT_WRIST)
    if not both_vis_ok(pose_lms, sh_idx, wr_idx):
        return False
    sh = pose_lms.landmark[sh_idx]; wr = pose_lms.landmark[wr_idx]
    return wr.y < (sh.y - raise_margin)

def shoulders_width_norm(pose_lms, w, h):
    L = pose_point_xy(pose_lms.landmark[POSE_LEFT_SHOULDER], w, h)
    R = pose_point_xy(pose_lms.landmark[POSE_RIGHT_SHOULDER], w, h)
    return float(np.linalg.norm(R - L) + 1e-8)

def wrists_distance_px(pose_lms, w, h):
    L = pose_point_xy(pose_lms.landmark[POSE_LEFT_WRIST], w, h)
    R = pose_point_xy(pose_lms.landmark[POSE_RIGHT_WRIST], w, h)
    return float(np.linalg.norm(R - L))

class ClapDetector:
    def __init__(self, close_thresh=0.30, open_thresh=0.48, min_drop_window=0.12, 
                 abs_close_px=90, abs_open_px=160, win=6, cooldown=0.35):
        self.close_thresh, self.open_thresh = close_thresh, open_thresh
        self.min_drop_window, self.abs_close_px = min_drop_window, abs_close_px
        self.abs_open_px, self.cooldown = abs_open_px, cooldown
        self.d_hist = deque(maxlen=win)
        self.state_open = True
        self.last_event_t = 0.0

    def update(self, pose_lms, w, h):
        now = time.time()
        event = False
        d_norm = float('nan'); drop = 0.0; hands_close = False
        if (pose_lms is None) or (not (both_vis_ok(pose_lms, POSE_LEFT_WRIST, POSE_RIGHT_WRIST) and
                                       both_vis_ok(pose_lms, POSE_LEFT_SHOULDER, POSE_RIGHT_SHOULDER))):
            self.d_hist.clear()
            return event, {"d_norm": d_norm, "hands_close": hands_close, "delta": drop}
        
        sw = shoulders_width_norm(pose_lms, w, h)
        d_px = wrists_distance_px(pose_lms, w, h)
        d_norm = (d_px / sw) if sw > 0 else float('nan')

        prev_vals = list(self.d_hist)
        prev_max = (max(prev_vals) if prev_vals else d_norm)
        drop = ((prev_max - d_norm) if (not np.isnan(d_norm) and not np.isnan(prev_max)) else 0.0)

        self.d_hist.append(d_norm if not np.isnan(d_norm) else 0.0)

        hands_close = ((not np.isnan(d_norm) and d_norm < self.close_thresh) or (d_px < self.abs_close_px))
        open_before = ((not np.isnan(prev_max) and prev_max > self.open_thresh) or (d_px > self.abs_open_px))

        cond_norm = ((not np.isnan(d_norm)) and (d_norm < self.close_thresh) and (drop > self.min_drop_window) and open_before)
        cond_abs = ((d_px < self.abs_close_px) and (drop > (self.min_drop_window * 0.6)) and open_before)

        if self.state_open and (cond_norm or cond_abs) and ((now - self.last_event_t) > self.cooldown):
            event, self.state_open, self.last_event_t = True, False, now

        if ((not np.isnan(d_norm) and d_norm > self.open_thresh) or (d_px > self.abs_open_px)):
            self.state_open = True
        
        return event, {"d_norm": float(d_norm), "hands_close": bool(hands_close), "delta": float(drop)}

def annotate_pose(frame, pose_lms, left_raised, right_raised):
    if pose_lms is None:
        return
    h, w = frame.shape[:2]
    lms = pose_lms.landmark
    pts = {}
    for idx in [POSE_LEFT_SHOULDER, POSE_RIGHT_SHOULDER, POSE_LEFT_ELBOW, POSE_RIGHT_ELBOW, 
                POSE_LEFT_WRIST, POSE_RIGHT_WRIST]:
        if lms[idx].visibility > min_vis:
            x = int(lms[idx].x * w); y = int(lms[idx].y * h); pts[idx] = (x, y)
    
    colorL = (0, 255, 0) if left_raised else (0, 0, 255)
    colorR = (0, 255, 0) if right_raised else (0, 0, 255)
    
    if POSE_LEFT_SHOULDER in pts: cv2.circle(frame, pts[POSE_LEFT_SHOULDER], 4, (255, 255, 0), -1)
    if POSE_RIGHT_SHOULDER in pts: cv2.circle(frame, pts[POSE_RIGHT_SHOULDER], 4, (255, 255, 0), -1)
    
    if POSE_LEFT_ELBOW in pts: cv2.circle(frame, pts[POSE_LEFT_ELBOW], 4, (200, 200, 255), -1)
    if POSE_RIGHT_ELBOW in pts: cv2.circle(frame, pts[POSE_RIGHT_ELBOW], 4, (200, 200, 255), -1)
    
    if POSE_LEFT_WRIST in pts: cv2.circle(frame, pts[POSE_LEFT_WRIST], 5, colorL, -1)
    if POSE_RIGHT_WRIST in pts: cv2.circle(frame, pts[POSE_RIGHT_WRIST], 5, colorR, -1)
    
    if POSE_LEFT_SHOULDER in pts and POSE_LEFT_ELBOW in pts:
        cv2.line(frame, pts[POSE_LEFT_SHOULDER], pts[POSE_LEFT_ELBOW], colorL, 2)
    if POSE_LEFT_ELBOW in pts and POSE_LEFT_WRIST in pts:
        cv2.line(frame, pts[POSE_LEFT_ELBOW], pts[POSE_LEFT_WRIST], colorL, 2)
    
    if POSE_RIGHT_SHOULDER in pts and POSE_RIGHT_ELBOW in pts:
        cv2.line(frame, pts[POSE_RIGHT_SHOULDER], pts[POSE_RIGHT_ELBOW], colorR, 2)
    if POSE_RIGHT_ELBOW in pts and POSE_RIGHT_WRIST in pts:
        cv2.line(frame, pts[POSE_RIGHT_ELBOW], pts[POSE_RIGHT_WRIST], colorR, 2)

def annotate_clap(frame, pose_lms, w, h, info, event=False):
    if pose_lms is None:
        return
    LW = pose_point_xy(pose_lms.landmark[POSE_LEFT_WRIST], w, h).astype(int)
    RW = pose_point_xy(pose_lms.landmark[POSE_RIGHT_WRIST], w, h).astype(int)
    color = (0, 255, 0) if info.get("hands_close", False) else (0, 0, 255)
    cv2.line(frame, tuple(LW), tuple(RW), color, 3 if info.get("hands_close", False) else 1)
    if event:
        cv2.circle(frame, ((LW[0] + RW[0]) // 2, (LW[1] + RW[1]) // 2), 12, (0, 255, 255), 2)
    def f2(x):
        try: return f"{float(x):.2f}" if x is not None else "nan"
        except: return "nan"
    txt = f"d_norm={f2(info.get('d_norm'))} drop={f2(info.get('delta'))}"
    cv2.putText(frame, txt, (10, 210), font, 0.6, (200, 255, 200), 2)


In [ ]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Не удалось открыть камеру")

fpt, ipz = init_mtcnn()
print(f"FPT initialized: {fpt is not None}")
print(f"IPZ initialized: {ipz is not None}")

clap_det = ClapDetector()
posture_det = PostureDetector(slouch_thresh=0.3)
clap_count = 0
fps_time = time.time()

while True:
    ok, frame = cap.read()
    if not ok: break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    h, w = frame.shape[:2]

    pose_res = pose.process(rgb)
    pose_lms = pose_res.pose_landmarks if pose_res and pose_res.pose_landmarks else None
    
    left_raised = is_hand_raised(pose_lms, left=True)
    right_raised = is_hand_raised(pose_lms, left=False)
    one_hand_raised = bool(left_raised ^ right_raised)
    two_hands_raised = bool(left_raised and right_raised)
    
    posture_status, posture_color, z_diff = posture_det.update(pose_lms)
    
    annotate_pose(frame, pose_lms, left_raised, right_raised)
    
    clap_event, clap_info = clap_det.update(pose_lms, w, h)
    if clap_event: clap_count += 1
    annotate_clap(frame, pose_lms, w, h, clap_info, event=clap_event)

    dets = mtcnn_detect(fpt, ipz, frame)
    if dets: draw_mtcnn_detections(frame, dets)

    smiling = False
    eyes_open = False
    fm = face_mesh.process(rgb)
    if fm.multi_face_landmarks:
        lms = fm.multi_face_landmarks[0].landmark
        smiling, sc = is_smile_facemesh(lms, w, h)
        eyes_open = are_eyes_open(lms, w, h)
        
        cv2.putText(frame, f"d={sc['d']:.3f} lift={sc['lift']:.3f} MAR={sc['mar']:.3f}",
                    (10, 30), font, 0.58, (255, 255, 0), 2)
        
        L = to_px(lms[LM_MOUTH_LEFT], w, h).astype(int)
        R = to_px(lms[LM_MOUTH_RIGHT], w, h).astype(int)
        A = to_px(lms[LM_UPPER_INNER], w, h).astype(int)
        B = to_px(lms[LM_LOWER_INNER], w, h).astype(int)
        cv2.line(frame, L, R, (255, 255, 0), 1)
        for p, color in [(L, (0, 255, 255)), (R, (0, 255, 255)), (A, (0, 140, 255)), (B, (0, 140, 255))]:
            cv2.circle(frame, tuple(p), 3, color, -1)
    else:
        best = max(dets, key=lambda d: d.get("confidence", 0.0)) if dets else None
        if best and "keypoints" in best:
            le = best["keypoints"].get("left_eye"); re = best["keypoints"].get("right_eye")
            ml = best["keypoints"].get("mouth_left"); mr = best["keypoints"].get("mouth_right")
            if None not in (le, re, ml, mr):
                iod = np.linalg.norm(np.array(le) - np.array(re)) + 1e-8
                mwr = np.linalg.norm(np.array(ml) - np.array(mr)) / iod
                eye_y = (le[1] + re[1]) / 2.0
                corner_y = (ml[1] + mr[1]) / 2.0
                lift2 = (eye_y - corner_y) / iod
                smiling = (mwr > 0.42) and (lift2 > 0.06)
                cv2.putText(frame, f"[MTCNN] MWR={mwr:.3f} LIFT2={lift2:.3f}",
                            (10, 30), font, 0.58, (200, 255, 200), 2)

    cv2.putText(frame, "SMILE" if smiling else "NO SMILE", (10, 60), font, 0.9, 
                (0, 255, 0) if smiling else (0, 0, 255), 2)
    cv2.putText(frame, f"EYES: {'OPEN' if eyes_open else 'CLOSED'}", (10, 85), font, 0.7, 
                (0, 255, 0) if eyes_open else (0, 0, 255), 2)
    cv2.putText(frame, f"POSTURE: {posture_status} (Z: {z_diff:.2f})", (10, 110), font, 0.7, 
                posture_color, 2)
    cv2.putText(frame, f"LEFT_RAISED: {left_raised}", (10, 135), font, 0.7, 
                (0, 255, 0) if left_raised else (0, 0, 255), 2)
    cv2.putText(frame, f"RIGHT_RAISED: {right_raised}", (10, 160), font, 0.7, 
                (0, 255, 0) if right_raised else (0, 0, 255), 2)
    cv2.putText(frame, f"ONE_HAND_RAISED (XOR): {one_hand_raised}", (10, 185), font, 0.7, 
                (0, 255, 0) if one_hand_raised else (0, 0, 255), 2)
    cv2.putText(frame, f"TWO_HANDS_RAISED: {two_hands_raised}", (10, 210), font, 0.7, 
                (0, 255, 0) if two_hands_raised else (0, 0, 255), 2)
    cv2.putText(frame, f"CLAPS: {clap_count}", (10, 235), font, 0.8, (0, 255, 255), 2)

    
    now = time.time()
    fps = 1.0 / max(now - fps_time, 1e-6)
    fps_time = now
    cv2.putText(frame, f"FPS:{fps:.1f}", (10, h - 10), font, 0.6, (180, 180, 180), 1)

    cv2.imshow("Hybrid: MTCNN + MediaPipe (Smile+Eyes+Posture+Hands+Clap)", frame)
    if (cv2.waitKey(1) & 0xFF) == 27:
        break

cap.release()
cv2.destroyAllWindows()

FPT initialized: True
IPZ initialized: False
